In [1]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, LSTM, Conv1D, GlobalMaxPooling1D, Dropout, Activation
from keras.layers import Input, TextVectorization
from keras.callbacks import *
from keras.metrics import *
from keras.backend import clear_session
from keras.layers import Embedding
from keras.initializers import Constant

import spacy

c:\Users\aryan\AppData\Local\Programs\Python\Python39\lib\site-packages\thinc\compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
c:\Users\aryan\AppData\Local\Programs\Python\Python39\lib\site-packages\thinc\compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


In [2]:
import pandas as pd
df = pd.read_csv('/Users/aryan/Actual-Coding/CDAC/NLI-TR (Turkish NLI Research) Dataset/snli_tr_train.csv')
df.head()

,idx,premise,hypothesis,label
0,0,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi atını yarışma için eğitiyor.,1
1,1,"Attaki bir kişi, bozuk bir uçağın üzerinden at...",Bir kişi bir lokantada omlet sipariş ediyor.,2
2,2,"Attaki bir kişi, bozuk bir uçağın üzerinden at...","Bir kişi açık havada, at üzerinde.",0
3,3,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Ailelerine gülümsüyorlar.,1
4,4,Fotoğraf makinesinde gülümseyen ve sallayan ço...,Burada çocuklar var.,0


In [3]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)

  return word_to_vec_map

In [4]:
word_to_vec_map = read_glove_vector('/Users/aryan/Actual-Coding/CDAC/NLI-TR (Turkish NLI Research) Dataset/glove.6B/glove.6B.100d.txt')

In [5]:
df.isnull().sum()

idx           0
premise       0
hypothesis    0
label         0
dtype: int64

In [6]:
df["label"].value_counts()

 0    183416
 2    183187
 1    182764
-1       785
Name: label, dtype: int64

In [7]:
X = df['hypothesis']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((385106,), (165046,), (385106,), (165046,))

In [8]:
tokenizer = Tokenizer()

#preparing vocabulary
tokenizer.fit_on_texts(list(X_train))
words_to_index = tokenizer.word_index

#converting text into integer sequences
X_train_seq  = tokenizer.texts_to_sequences(X_train) 
X_test_seq = tokenizer.texts_to_sequences(X_test)

#padding to prepare sequences of same length
X_train_seq  = pad_sequences(X_train_seq, maxlen=100)
X_test_seq = pad_sequences(X_test_seq, maxlen=100)

In [9]:
X_train_seq.shape

(385106, 100)

In [10]:
X_test_seq.shape

(165046, 100)

In [11]:
size_of_vocabulary=len(tokenizer.word_index) + 1
print(size_of_vocabulary)

54020


In [12]:
METRICS = [
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR'),
]

In [13]:
size_of_vocabulary=len(tokenizer.word_index) + 1
size_of_vocabulary

54020

In [14]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]

emb_matrix = np.zeros((100, #vocab_len
                       embed_vector_len))

for word, index in words_to_index.items():
  embedding_vector = word_to_vec_map.get(word)
  if embedding_vector is not None:
    emb_matrix[index, :] = embedding_vector

embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=size_of_vocabulary, weights = [emb_matrix], trainable=False)

IndexError: index 100 is out of bounds for axis 0 with size 100

In [ ]:
vocab_len

54019

In [ ]:
embed_vector_len

100

In [ ]:
embedding_layer.output_shape

(None, 54020, 100)

In [ ]:
clear_session()

model=Sequential()

#embedding layer
model.add(embedding_layer) 
model.add(Dense(64, input_shape=(100,))) #error at epochs

#lstm layer
model.add(LSTM(128,return_sequences=True,dropout=0.2))

#Global Maxpooling
model.add(GlobalMaxPooling1D())

#Dense Layer
model.add(Dense(64,activation='relu')) 
model.add(Dense(1,activation='sigmoid')) 

#Add loss function, metrics, optimizer
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=METRICS) 

#Adding callbacks
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)  
mc=ModelCheckpoint('best_model', monitor='val_loss', mode='min', save_best_only=True, save_weights_only=False, verbose=1)  

#Print summary of model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54020, 100)        5401900   
                                                                 
 dense (Dense)               (None, 54020, 64)         6464      
                                                                 
 lstm (LSTM)                 (None, 54020, 128)        98816     
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                        

#Epochs! Here we go again, to fail

In [ ]:
history = model.fit(np.array(X_train_seq),np.array(y_train),batch_size=128,epochs=10,
                    validation_data=(np.array(X_test_seq),np.array(y_test)),verbose=1,callbacks=[es,mc]) # type: ignore

Epoch 1/10


ValueError: in user code:

    File "c:\Users\aryan\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\aryan\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\aryan\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\aryan\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\aryan\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\aryan\App

#InvalidArgumentError:  I have No clue how to fix it.

indices[39,99] = 54019 is not in [0, 54019)
 [[node sequential/embedding/embedding_lookup (defined at tmp/ipykernel_27/1248510687.py:2) ]] [Op:__inference_train_function_4417]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/embedding/embedding_lookup:
 sequential/embedding/embedding_lookup/2605 (defined at opt/conda/lib/python3.7/contextlib.py:112)

In [ ]:
model_results = model.evaluate(X_test_seq, y_test, batch_size=128, verbose=0) # type: ignore

for name, value in zip(model.metrics_names, model_results):
  print(name, ': ', value)